In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Sep 24 13:16:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

%matplotlib inline

In [3]:
dtype = 'float32'
tf.keras.backend.set_floatx(dtype)

In [4]:
cifar10 = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.astype(dtype) / 255.0
y_train = y_train.astype(dtype)
X_test = X_test.astype(dtype)  / 255.0
y_test = y_test.astype(dtype)

X_train = np.reshape(X_train, (-1, 3072))
X_test = np.reshape(X_test, (-1, 3072))

X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

170508288/170498071 [==============================] - 4s 0us/step


In [32]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_norm = scaler.transform(X)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

X_train_norm = np.reshape(X_train_norm, (-1, 32, 32, 3))
X_test_norm = np.reshape(X_test_norm, (-1, 32, 32, 3))

In [84]:
class Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, regularization_penalty, regularization_method):
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method

    def __call__(self, x):
        if self.regularization_method == 'weighted_l1':
            return self.weighted_l1(x)
        elif self.regularization_method == 'group_sparsity':
            return self.group_sparsity(x)
        else:
            raise NotImplementedError(f"Unknown regularization method {self.regularization_method}")
    
    def weighted_l1(self, x):
        # I.e. for a parameter matrix of 4 input and 10 output neurons:
        #
        # [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]
        #
        # the scaling tensor, as well as the resulting weighted values, could be:
        #
        # [[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]]
        #
        # Therefore every additional output neuron is regularized more.

        scaling_tensor = tf.cumsum(tf.constant(self.regularization_penalty, shape=x.shape, dtype=dtype), axis=-1)
        weighted_values = scaling_tensor * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def group_sparsity(self, x):
        # I.e. for a parameter matrix of 3 input and 5 output neurons:
        #
        # [[1., 1., 1., 1., 1.],
        #  [1., 2., 2., 1., 2.],
        #  [2., 2., 3., 1., 3.]]
        #
        # The resulting vector of group norms is [2., 2., 3., 1., 3.], therefore for
        # every output neuron, its incoming connections form a group.

        # TODO implement for Conv2D layers
        group_norms = tf.norm(x, ord=2, axis=0)
        # assert group_norms.shape[0] == x.shape[1]
        return self.regularization_penalty * tf.reduce_sum(group_norms)

    def get_config(self):
        return {'regularization_penalty': float(self.regularization_penalty)}


class ModelReference:
    def __init__(self, model):
        self.model = model


class CustomLayer(tf.keras.Model):
    def __init__(self, input_shape):
        super().__init__()

        self.inpt_shp = input_shape
    
    def configure(self, model):
        self.mr = ModelReference(model)
    
    def get_input_shape(self):
        if self.inpt_shp is not None:
            return self.inpt_shp
        
        return self.mr.model.get_layer_input_shape(self)
    
    def get_output_shape(self):
        return self.mr.model.get_layer_output_shape(self)


class Dense(CustomLayer):
    def __init__(self, units, activation, regularization_penalty=0.01, 
                 regularization_method='weighted_l1', kernel_initializer='glorot_uniform', 
                 bias_initializer='zeros', input_shape=None, fixed_size=False):
        super().__init__(input_shape)

        self.units = units
        self.activation = activation
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        self.fixed_size = fixed_size
        
        self.A = tf.keras.activations.get(activation)
        self.W_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = Regularizer(self.regularization_penalty, self.regularization_method)
    
    def configure(self, model):
        super().configure(model)

        input_units = self.get_input_units_count()

        self.W = tf.Variable(
            name='W',
            initial_value=self.W_init(shape=(input_units, self.units), dtype=dtype),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(self.units,), dtype=dtype),
            trainable=True)
        
        if self.regularization_method is not None:
            self.add_loss(lambda: self.regularizer(tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)))
    
    def call(self, inputs):
        return self.A(tf.matmul(inputs, self.W) + self.b)
    
    def copy_without_regularization(self):
        # TODO fix
        copy = Dense(
            self.input_units, 
            self.units, 
            self.activation, 
            regularization_penalty=self.regularization_penalty, 
            regularization_method=None, 
            kernel_initializer=self.kernel_initializer, 
            bias_initializer=self.bias_initializer
        )
        copy.W = self.W
        copy.b = self.b
        return copy
    
    def get_input_units_count(self):
        input_shape = self.get_input_shape()
        if len(input_shape) != 1:
            raise Exception(f"Invalid input shape {input_shape}.")
        return input_shape[0]
    
    def get_size(self):
        return self.get_input_units_count(), self.W.shape[1]
    
    def prune(self, threshold, active_input_units_indices):
        # Remove connections from pruned units in previous layer
        new_W = tf.gather(self.W.value(), active_input_units_indices, axis=0)

        if self.fixed_size:
            active_output_neurons_indices = list(range(new_W.shape[1]))
        else:
            # Prune units in this layer
            weights_with_biases = tf.concat([new_W, tf.reshape(self.b.value(), (1, -1))], axis=0)
            neurons_are_active = tf.math.reduce_max(tf.abs(weights_with_biases), axis=0) >= threshold
            active_output_neurons_indices = tf.reshape(tf.where(neurons_are_active), (-1,))
            
            new_W = tf.gather(new_W, active_output_neurons_indices, axis=1)
            new_b = tf.gather(self.b.value(), active_output_neurons_indices, axis=0)

            self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.W = tf.Variable(name='W', initial_value=new_W, trainable=True)

        return active_output_neurons_indices
    
    def grow(self, n_new_input_units, percentage, min_new_units, scaling_factor):
        if n_new_input_units > 0:
            # Add connections to grown units in previous layer
            W_growth = self.W_init(shape=(self.W.shape[0] + n_new_input_units, self.W.shape[1]), dtype=dtype)[-n_new_input_units:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.
            new_W = tf.concat([self.W.value(), W_growth], axis=0)
        else:
            new_W = self.W.value()

        if self.fixed_size:
            n_new_output_units = 0
        else:
            # Grow new units in this layer
            n_new_output_units = max(min_new_units, int(new_W.shape[1] * percentage))
            W_growth = self.W_init(shape=(new_W.shape[0], new_W.shape[1] + n_new_output_units), dtype=dtype)[:, -n_new_output_units:] * scaling_factor
            b_growth = self.b_init(shape=(n_new_output_units,), dtype=dtype)  # TODO for all possible bias initializers to work properly, the whole bias vector should be initialized at once
            new_W = tf.concat([new_W, W_growth], axis=1)
            new_b = tf.concat([self.b.value(), b_growth], axis=0)

            self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.W = tf.Variable(name='W', initial_value=new_W, trainable=True)

        return n_new_output_units
    
    def get_param_string():
        param_string = ""
        weights_with_bias = tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)
        max_parameters = tf.math.reduce_max(tf.abs(weights_with_bias), axis=0).numpy()
        magnitudes = np.floor(np.log10(max_parameters))
        for m in magnitudes:
            if m > 0:
                m = 0
            param_string += str(int(-m))
        return param_string


class Conv2D(CustomLayer):
    def __init__(self, filters, filter_size, activation, strides=(1, 1), 
                 padding='SAME', regularization_penalty=0.01, 
                 regularization_method='weighted_l1', kernel_initializer='glorot_uniform',
                 bias_initializer='zeros', input_shape=None, fixed_size=False):
        super().__init__(input_shape)
    
        self.filters = filters
        self.filter_size = filter_size
        self.activation = activation
        self.strides = strides
        self.padding = padding
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        self.fixed_size = fixed_size
        
        self.A = tf.keras.activations.get(activation)
        self.F_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = Regularizer(self.regularization_penalty, self.regularization_method)
    
    def configure(self, model):
        super().configure(model)

        input_filters = self.get_input_filters_count()

        self.F = tf.Variable(
            name='F',
            initial_value=self.F_init(
                shape=(self.filter_size[0], self.filter_size[1], input_filters, self.filters), dtype=dtype
            ),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(self.filters,), dtype=dtype),
            trainable=True)

        if self.regularization_method is not None:
            self.add_loss(lambda: self.regularizer(tf.concat([tf.reshape(self.F, (-1, self.F.shape[-1])), tf.reshape(self.b, (1, -1))], axis=0)))
    
    def call(self, inputs):
        y = tf.nn.conv2d(inputs, self.F, strides=self.strides, padding=self.padding)
        y = tf.nn.bias_add(y, self.b)
        y = self.A(y)
        return y
    
    def copy_without_regularization(self):
        # TODO fix
        copy = Conv2D(
            self.input_filters,
            self.filters,
            self.filter_size,
            self.activation, 
            strides=self.strides, 
            padding=self.padding, 
            kernel_initializer=self.kernel_initializer, 
            bias_initializer=self.bias_initializer 
        )
        copy.F = self.F
        copy.b = self.b
        return copy
    
    def get_input_filters_count(self):
        input_shape = self.get_input_shape()
        return input_shape[-1]
    
    def get_size(self):
        return self.get_input_filters_count(), self.F.shape[-1]
    
    def prune(self, threshold, active_input_units_indices):
        # Remove connections from pruned units in previous layer
        new_F = tf.gather(self.F.value(), active_input_units_indices, axis=-2)

        if self.fixed_size:
            active_output_filters_indices = list(range(new_F.shape[-1]))
        else:
            # Prune units in this layer
            F_reduced_max = tf.reshape(tf.math.reduce_max(tf.abs(new_F), axis=(0, 1, 2)), (1, -1))
            F_reduced_max_with_biases = tf.concat([F_reduced_max, tf.reshape(self.b.value(), (1, -1))], axis=0)
            filters_are_active = tf.math.reduce_max(tf.abs(F_reduced_max_with_biases), axis=0) >= threshold
            active_output_filters_indices = tf.reshape(tf.where(filters_are_active), (-1,))
            
            new_F = tf.gather(new_F, active_output_filters_indices, axis=-1)
            new_b = tf.gather(self.b.value(), active_output_filters_indices, axis=0)

            self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.F = tf.Variable(name='F', initial_value=new_F, trainable=True)

        return active_output_filters_indices

    def grow(self, n_new_input_units, percentage, min_new_units, scaling_factor):
        if n_new_input_units > 0:
            # Add connections to grown units in previous layer
            F_growth = self.F_init(shape=(self.F.shape[0], self.F.shape[1], self.F.shape[2] + n_new_input_units, self.F.shape[3]), dtype=dtype)[:, :, -n_new_input_units:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.
            new_F = tf.concat([self.F.value(), F_growth], axis=-2)
        else:
            new_F = self.F.value()

        if self.fixed_size:
            n_new_output_units = 0
        else:
            # Grow new units in this layer
            n_new_output_units = max(min_new_units, int(new_F.shape[-1] * percentage))
            F_growth = self.F_init(shape=(new_F.shape[0], new_F.shape[1], new_F.shape[2], new_F.shape[3] + n_new_output_units), dtype=dtype)[:, :, :, -n_new_output_units:] * scaling_factor
            b_growth = self.b_init(shape=(n_new_output_units,), dtype=dtype)  # TODO for all possible bias initializers to work properly, the whole bias vector should be initialized at once
            new_F = tf.concat([new_F, F_growth], axis=-1)
            new_b = tf.concat([self.b.value(), b_growth], axis=0)

            self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.F = tf.Variable(name='F', initial_value=new_F, trainable=True)

        return n_new_output_units

    def get_param_string():
        param_string = ""
        # TODO
        return param_string


class Flatten(tf.keras.Model):
    def call(self, inputs):
        return tf.reshape(tf.transpose(inputs, perm=[0, 3, 1, 2]), (inputs.shape[0], -1))


class Sequential(tf.keras.Model):
    def __init__(self, layers, activation=None):
        super().__init__()
        
        self.lrs = list()
        for layer in layers:
            self.lrs.append(layer)
            if isinstance(layer, CustomLayer):
                layer.configure(self)

    def call(self, inputs):
        x = inputs
        for layer in self.lrs:
            x = layer(x)
        return x
    
    def get_layer_input_shape(self, target_layer):
        input = np.random.normal(size=(1,) + self.lrs[0].inpt_shp)
        for layer in self.lrs:
            if layer is target_layer:
                return tuple(input.shape[1:])
            input = layer(input)
        raise Exception("Layer not found in the model.")

    def get_layer_output_shape(self, target_layer):
        input = np.random.normal(size=(1,) + self.lrs[0].inpt_shp)
        for layer in self.lrs:
            output = layer(input)
            if layer is target_layer:
                return tuple(output.shape[1:])
            input = output
        raise Exception("Layer not found in the model.")
    
    def get_layer_sizes(self):
        """
        Returns the sizes of all layers in the model, including the input and output layer.
        """
        layer_sizes = list()
        first_layer = True
        for l in range(len(self.lrs)):
            layer = self.lrs[l]
            if isinstance(layer, CustomLayer):
                layer_size = layer.get_size()
                if first_layer:
                    layer_sizes.append(layer_size[0])
                    first_layer = False
                layer_sizes.append(layer_size[1])
        return layer_sizes
    
    def get_hidden_layer_sizes(self):
        return self.get_layer_sizes()[1:-1]
    
    def remove_regularization(self):
        for l in range(len(self.lrs)):
            layer = self.lrs[l]
            if isinstance(layer, CustomLayer):
                self.lrs[l] = layer.copy_without_regularization()
    
    def get_regularization_penalty(self):
        return self.lrs[-2].regularizer.regularization_penalty
    
    def set_regularization_penalty(self, regularization_penalty):
        for layer in self.lrs:
            if isinstance(layer, CustomLayer) and not layer.fixed_size:
                layer.regularizer.regularization_penalty = regularization_penalty
    
    def prune(self, threshold=0.001):
        # for l in range(len(self.lrs) - 1):
        #     layer1 = self.lrs[l]
        #     layer2 = self.lrs[l + 1]
            
        #     W1 = layer1.W.value()
        #     b1 = layer1.b.value()
        #     W2 = layer2.W.value()

        #     weights_with_biases = tf.concat([W1, tf.reshape(b1, (1, -1))], axis=0)
        #     neurons_are_active = tf.math.reduce_max(tf.abs(weights_with_biases), axis=0) >= threshold
        #     active_neurons_indices = tf.reshape(tf.where(neurons_are_active), (-1,))
            
        #     new_W1 = tf.gather(W1, active_neurons_indices, axis=1)
        #     new_b1 = tf.gather(b1, active_neurons_indices, axis=0)
        #     new_W2 = tf.gather(W2, active_neurons_indices, axis=0)

        #     layer1.W = tf.Variable(name='W', initial_value=new_W1, trainable=True)
        #     layer1.b = tf.Variable(name='b', initial_value=new_b1, trainable=True)
        #     layer2.W = tf.Variable(name='W', initial_value=new_W2, trainable=True)
        input_shape = self.lrs[0].get_input_shape()
        n_input_units = input_shape[-1]
        active_units_indices = list(range(n_input_units))

        last_custom_layer = None
        for layer in self.lrs:
            if isinstance(layer, CustomLayer):
                if last_custom_layer is not None and type(last_custom_layer) != type(layer):
                    if type(last_custom_layer) == Conv2D and type(layer) == Dense:
                        convolutional_shape = last_custom_layer.get_output_shape()
                        active_units_indices = self.convert_channel_indices_to_flattened_indices(active_units_indices, convolutional_shape)
                    else:
                        raise Exception("Incorrect order of custom layer types.")
                active_units_indices = layer.prune(threshold, active_units_indices)
                last_custom_layer = layer
    
    def grow(self, percentage, min_new_neurons=5, scaling_factor=0.001):   
        # for l in range(len(self.lrs) - 1):
        #     layer1 = self.lrs[l]
        #     layer2 = self.lrs[l + 1]
       
        #     W1 = layer1.W.value()
        #     b1 = layer1.b.value()
        #     W2 = layer2.W.value()

        #     n_new_neurons = max(min_new_neurons, int(W1.shape[1] * percentage))

        #     W1_growth = layer1.W_init(shape=(W1.shape[0], W1.shape[1] + n_new_neurons), dtype=dtype)[:, -n_new_neurons:] * scaling_factor
        #     b1_growth = layer1.b_init(shape=(n_new_neurons,), dtype=dtype)
        #     W2_growth = layer2.W_init(shape=(W2.shape[0] + n_new_neurons, W2.shape[1]), dtype=dtype)[-n_new_neurons:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.

        #     new_W1 = tf.concat([W1, W1_growth], axis=1)
        #     new_b1 = tf.concat([b1, b1_growth], axis=0)
        #     new_W2 = tf.concat([W2, W2_growth], axis=0)

        #     layer1.W = tf.Variable(name='W1', initial_value=new_W1, trainable=True)
        #     layer1.b = tf.Variable(name='b1', initial_value=new_b1, trainable=True)
        #     layer2.W = tf.Variable(name='W2', initial_value=new_W2, trainable=True)
        n_new_units = 0

        last_custom_layer = None
        for layer in self.lrs:
            if isinstance(layer, CustomLayer):
                if last_custom_layer is not None and type(last_custom_layer) != type(layer):
                    if type(last_custom_layer) == Conv2D and type(layer) == Dense:
                        convolutional_shape = last_custom_layer.get_output_shape()
                        n_new_units = n_new_units * convolutional_shape[0] * convolutional_shape[1]
                    else:
                        raise Exception("Incorrect order of custom layer types.")
                n_new_units = layer.grow(n_new_units, percentage, min_new_units=min_new_neurons, scaling_factor=scaling_factor)
                last_custom_layer = layer
    
    @staticmethod
    def convert_channel_indices_to_flattened_indices(channel_indices, convolutional_shape):
        dense_indices = list()
        units_per_channel = convolutional_shape[0] * convolutional_shape[1]
        for channel_index in channel_indices:
            for iter in range(units_per_channel):
                dense_indices.append(channel_index * units_per_channel + iter)
        return dense_indices
    
    def print_neurons(self):
        for layer in self.lrs[:-1]:
            print(layer.get_param_string())
    
    def evaluate(self, x, y, summed_training_loss, summed_training_accuracy, val_dataset):
        # Calculate training loss and accuracy
        if summed_training_loss is not None:
            loss = summed_training_loss / x.shape[0]
        else:
            loss = None
        
        if summed_training_accuracy is not None:
            accuracy = summed_training_accuracy / x.shape[0]
        else:
            accuracy = None
        
        # Calculate val loss and accuracy
        summed_val_loss = 0
        summed_val_accuracy = 0
        n_val_instances = 0
        
        for step, (x_batch, y_batch) in enumerate(val_dataset):
            y_pred = self(x_batch)
            summed_val_loss += tf.reduce_sum(tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred))
            summed_val_accuracy += float(tf.reduce_sum(tf.keras.metrics.sparse_categorical_accuracy(y_batch, y_pred)))
            n_val_instances += x_batch.shape[0]
        
        val_loss = summed_val_loss / n_val_instances
        val_accuracy = summed_val_accuracy / n_val_instances

        return loss, accuracy, val_loss, val_accuracy
    
    def print_epoch_statistics(self, x, y, summed_training_loss, summed_training_accuracy, val_dataset, print_neurons):
        loss, accuracy, val_loss, val_accuracy = self.evaluate(x, y, summed_training_loss, summed_training_accuracy, val_dataset)
        print(f"loss: {loss} - accuracy: {accuracy} - val_loss: {val_loss} - val_accuracy: {val_accuracy} - penalty: {model.get_regularization_penalty()}")
        hidden_layer_sizes = self.get_hidden_layer_sizes()
        print(f"hidden layer sizes: {hidden_layer_sizes}, total units: {sum(hidden_layer_sizes)}")
        if print_neurons:
            self.print_neurons()
    
    def update_history(self, x, y, summed_loss, summed_accuracy, val_dataset, history):
        loss, accuracy, val_loss, val_accuracy = self.evaluate(x, y, summed_loss, summed_accuracy, val_dataset)
        history['loss'].append(loss)
        history['accuracy'].append(accuracy)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_accuracy)

    def fit(self, x, y, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data, pruning_threshold=0.001, 
            regularization_penalty_multiplier=1., stall_coefficient=1, growth_percentage=0.2, mini_epochs_per_epoch=1, verbose=True, print_neurons=False):
        train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
        train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

        val_dataset = tf.data.Dataset.from_tensor_slices(validation_data).batch(batch_size)

        history = {
            'loss': list(),
            'accuracy': list(),
            'val_loss': list(),
            'val_accuracy': list(),
        }

        best_val_loss = np.inf
        training_stalled = False
        for epoch in range(epochs):
            summed_loss = 0
            summed_accuracy = 0

            if verbose:
                print("##########################################################")
                print(f"Epoch {epoch + 1}/{epochs}")

            if epoch < self_scaling_epochs:
                if verbose:
                    print("Before growing:")
                    self.print_epoch_statistics(x, y, None, None, val_dataset, print_neurons)

                loss, accuracy, val_loss, val_accuracy = self.evaluate(x, y, summed_loss, summed_accuracy, val_dataset)
                if regularization_penalty_multiplier != 1. and val_loss >= best_val_loss * stall_coefficient:
                    if not training_stalled:
                        penalty = self.get_regularization_penalty() * regularization_penalty_multiplier
                        print("Changing penalty...")
                        # TODO this must be modified, penalty can differ for each layer
                        self.set_regularization_penalty(penalty)
                        training_stalled = True
                else:
                    best_val_loss = val_loss
                    training_stalled = False

                self.grow(percentage=growth_percentage, min_new_neurons=min_new_neurons, scaling_factor=pruning_threshold)
                if verbose:
                    print("After growing:")
                    self.print_epoch_statistics(x, y, None, None, val_dataset, print_neurons)
            
            if epoch == self_scaling_epochs and self_scaling_epochs > 0:
                self.remove_regularization()

            for mini_epoch in range(mini_epochs_per_epoch):
                for step, (x_batch, y_batch) in enumerate(train_dataset):
                    with tf.GradientTape() as tape:
                        y_pred = self(x_batch, training=True)
                        raw_loss = tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred)
                        loss_value = tf.reduce_mean(raw_loss)
                        loss_value += sum(self.losses)  # Add losses registered by model.add_loss

                        summed_loss += tf.reduce_sum(raw_loss)
                        summed_accuracy += float(tf.reduce_sum(tf.keras.metrics.sparse_categorical_accuracy(y_batch, y_pred)))

                    grads = tape.gradient(loss_value, self.trainable_variables)
                    optimizer.apply_gradients(zip(grads, self.trainable_variables))
            
            if epoch < self_scaling_epochs:
                if verbose:
                    print("Before pruning:")
                    self.print_epoch_statistics(x, y, summed_loss, summed_accuracy, val_dataset, print_neurons)
                self.prune(threshold=pruning_threshold)
                if verbose:
                    print("After pruning:")
                    self.print_epoch_statistics(x, y, None, None, val_dataset, print_neurons)
            else:
                if verbose:
                    self.print_epoch_statistics(x, y, summed_loss, summed_accuracy, val_dataset, print_neurons)
            
            self.update_history(x, y, summed_loss, summed_accuracy, val_dataset, history)

        return history

# Convolutions

In [63]:
F = tf.cumsum(tf.cumsum(tf.ones(shape=(2, 3, 3, 5), dtype=dtype), axis=-1), axis=0)
b = tf.constant(0.1, shape=(5,), dtype=dtype)
new_F = F

In [65]:
F

<tf.Tensor: shape=(2, 3, 3, 5), dtype=float32, numpy=
array([[[[ 1.,  2.,  3.,  4.,  5.],
         [ 1.,  2.,  3.,  4.,  5.],
         [ 1.,  2.,  3.,  4.,  5.]],

        [[ 1.,  2.,  3.,  4.,  5.],
         [ 1.,  2.,  3.,  4.,  5.],
         [ 1.,  2.,  3.,  4.,  5.]],

        [[ 1.,  2.,  3.,  4.,  5.],
         [ 1.,  2.,  3.,  4.,  5.],
         [ 1.,  2.,  3.,  4.,  5.]]],


       [[[ 2.,  4.,  6.,  8., 10.],
         [ 2.,  4.,  6.,  8., 10.],
         [ 2.,  4.,  6.,  8., 10.]],

        [[ 2.,  4.,  6.,  8., 10.],
         [ 2.,  4.,  6.,  8., 10.],
         [ 2.,  4.,  6.,  8., 10.]],

        [[ 2.,  4.,  6.,  8., 10.],
         [ 2.,  4.,  6.,  8., 10.],
         [ 2.,  4.,  6.,  8., 10.]]]], dtype=float32)>

In [40]:
tf.reshape(F, (-1, 5))

<tf.Tensor: shape=(18, 5), dtype=float32, numpy=
array([[1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.]], dtype=float32)>

In [66]:
tf.cumsum(tf.constant(0.1, shape=F.shape, dtype=dtype), axis=-1)

<tf.Tensor: shape=(2, 3, 3, 5), dtype=float32, numpy=
array([[[[0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5]],

        [[0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5]],

        [[0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5]]],


       [[[0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5]],

        [[0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5]],

        [[0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5]]]], dtype=float32)>

In [71]:
flattened = tf.reshape(tf.transpose(F, perm=[0, 3, 1, 2]), (F.shape[0], -1))
flattened

<tf.Tensor: shape=(2, 45), dtype=float32, numpy=
array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,  2.,  2.,
         2.,  2.,  2.,  2.,  2.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
         3.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  5.,  5.,  5.,
         5.,  5.,  5.,  5.,  5.,  5.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  4.,  4.,  4.,  4.,
         4.,  4.,  4.,  4.,  4.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
         6.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8., 10., 10., 10.,
        10., 10., 10., 10., 10., 10.]], dtype=float32)>

In [72]:
input = [1, 2, 4]
output = list()
for inp in input:
    for iter in range(3*3):
        output.append(inp * 3*3 + iter)
output

[9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44]

In [73]:
tf.gather(flattened, output, axis=-1)

<tf.Tensor: shape=(2, 27), dtype=float32, numpy=
array([[ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  3.,  3.,  3.,
         3.,  3.,  3.,  3.,  3.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
         5.],
       [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  6.,  6.,  6.,  6.,
         6.,  6.,  6.,  6.,  6., 10., 10., 10., 10., 10., 10., 10., 10.,
        10.]], dtype=float32)>

In [55]:
tf.reshape(F, (-1,))

<tf.Tensor: shape=(90,), dtype=float32, numpy=
array([1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2.,
       3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4.,
       5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1.,
       2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3.,
       4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5.,
       1., 2., 3., 4., 5.], dtype=float32)>

In [53]:
flatten = tf.keras.layers.Flatten(data_format='channels_last')
flattened = flatten(F)
flattened

<tf.Tensor: shape=(2, 45), dtype=float32, numpy=
array([[1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1.,
        2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2.,
        3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1.,
        2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2.,
        3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5.]],
      dtype=float32)>

In [16]:
flattened.shape

TensorShape([2, 45])

In [17]:
3*3*5

45

In [19]:
input = [1, 2, 4]
output = list()
for iter in range(3*3):
    for inp in input:
        output.append(inp + iter * 5)
output

[1,
 2,
 4,
 6,
 7,
 9,
 11,
 12,
 14,
 16,
 17,
 19,
 21,
 22,
 24,
 26,
 27,
 29,
 31,
 32,
 34,
 36,
 37,
 39,
 41,
 42,
 44]

In [22]:
flattened[0]

<tf.Tensor: shape=(45,), dtype=float32, numpy=
array([1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2.,
       3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5., 1., 2., 3., 4.,
       5., 1., 2., 3., 4., 5., 1., 2., 3., 4., 5.], dtype=float32)>

In [25]:
tf.gather(flattened, output, axis=-1)

<tf.Tensor: shape=(2, 27), dtype=float32, numpy=
array([[2., 3., 5., 2., 3., 5., 2., 3., 5., 2., 3., 5., 2., 3., 5., 2.,
        3., 5., 2., 3., 5., 2., 3., 5., 2., 3., 5.],
       [2., 3., 5., 2., 3., 5., 2., 3., 5., 2., 3., 5., 2., 3., 5., 2.,
        3., 5., 2., 3., 5., 2., 3., 5., 2., 3., 5.]], dtype=float32)>

In [ ]:
F_reduced_max = tf.reshape(tf.math.reduce_max(tf.abs(F), axis=(0, 1, 2)), (1, -1))
F_reduced_max_with_biases = tf.concat([F_reduced_max, tf.reshape(b, (1, -1))], axis=0)
filters_are_active = tf.math.reduce_max(tf.abs(F_reduced_max_with_biases), axis=0) >= 3
filters_are_active

<tf.Tensor: shape=(5,), dtype=bool, numpy=array([False, False,  True,  True,  True])>

In [ ]:
b

<tf.Tensor: shape=(1, 1, 2, 5), dtype=float32, numpy=
array([[[[0.1, 0.2, 0.3, 0.4, 0.5],
         [0.1, 0.2, 0.3, 0.4, 0.5]]]], dtype=float32)>

InvalidArgumentError: ignored

In [ ]:
weights_with_biases = F
filters_are_active = tf.math.reduce_max(tf.abs(weights_with_biases), axis=(0, 1, 2)) >= 3
active_output_filters_indices = tf.reshape(tf.where(filters_are_active), (-1,))
new_F = tf.gather(F, active_output_neurons_indices, axis=-1)
new_b = tf.gather(b, active_output_neurons_indices, axis=0)

In [ ]:
new_F

<tf.Tensor: shape=(3, 3, 2, 3), dtype=float32, numpy=
array([[[[3., 4., 5.],
         [3., 4., 5.]],

        [[3., 4., 5.],
         [3., 4., 5.]],

        [[3., 4., 5.],
         [3., 4., 5.]]],


       [[[3., 4., 5.],
         [3., 4., 5.]],

        [[3., 4., 5.],
         [3., 4., 5.]],

        [[3., 4., 5.],
         [3., 4., 5.]]],


       [[[3., 4., 5.],
         [3., 4., 5.]],

        [[3., 4., 5.],
         [3., 4., 5.]],

        [[3., 4., 5.],
         [3., 4., 5.]]]], dtype=float32)>

In [ ]:
new_b

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.1, 0.1, 0.1], dtype=float32)>

In [ ]:
w = tf.cumsum(x, axis=-2)
w

<tf.Tensor: shape=(3, 3, 4, 2), dtype=float32, numpy=
array([[[[1., 1.],
         [2., 2.],
         [3., 3.],
         [4., 4.]],

        [[1., 1.],
         [2., 2.],
         [3., 3.],
         [4., 4.]],

        [[1., 1.],
         [2., 2.],
         [3., 3.],
         [4., 4.]]],


       [[[1., 1.],
         [2., 2.],
         [3., 3.],
         [4., 4.]],

        [[1., 1.],
         [2., 2.],
         [3., 3.],
         [4., 4.]],

        [[1., 1.],
         [2., 2.],
         [3., 3.],
         [4., 4.]]],


       [[[1., 1.],
         [2., 2.],
         [3., 3.],
         [4., 4.]],

        [[1., 1.],
         [2., 2.],
         [3., 3.],
         [4., 4.]],

        [[1., 1.],
         [2., 2.],
         [3., 3.],
         [4., 4.]]]], dtype=float32)>

In [ ]:
active_input_neurons_indices = (1, 3)

In [ ]:
tf.gather(w, active_input_neurons_indices, axis=-2)

<tf.Tensor: shape=(3, 3, 2, 2), dtype=float32, numpy=
array([[[[2., 2.],
         [4., 4.]],

        [[2., 2.],
         [4., 4.]],

        [[2., 2.],
         [4., 4.]]],


       [[[2., 2.],
         [4., 4.]],

        [[2., 2.],
         [4., 4.]],

        [[2., 2.],
         [4., 4.]]],


       [[[2., 2.],
         [4., 4.]],

        [[2., 2.],
         [4., 4.]],

        [[2., 2.],
         [4., 4.]]]], dtype=float32)>

In [ ]:
x = X_train_norm[:2, :, :, :]

In [ ]:
x = np.array([[
      [[0, 1, 2], [0, 1, 2]],
      [[0, 1, 2], [0, 1, 2]],
]])

In [ ]:
x.shape

(1, 2, 2, 3)

In [ ]:
flatten = tf.keras.layers.Flatten(data_format='channels_last')
y = flatten(x)
y

<tf.Tensor: shape=(1, 12), dtype=int64, numpy=array([[0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]])>

In [ ]:
y.shape

TensorShape([2, 3072])

In [44]:
epochs = 20
self_scaling_epochs = 20
batch_size = 256
min_new_neurons = 20

In [31]:
model = Sequential([
        Dense(20, activation='selu', regularization_penalty=0.000001, 
            regularization_method='weighted_l1', kernel_initializer='lecun_normal', input_shape=X_train_norm[0, :].shape),
        Dense(20, activation='selu', regularization_penalty=0.000001, 
            regularization_method='weighted_l1', kernel_initializer='lecun_normal'),
        Dense(20, activation='selu', regularization_penalty=0.000001, 
            regularization_method='weighted_l1', kernel_initializer='lecun_normal'),
        Dense(20, activation='selu', regularization_penalty=0.000001, 
            regularization_method='weighted_l1', kernel_initializer='lecun_normal'),
        Dense(10, activation='softmax', regularization_penalty=0., 
            regularization_method=None, fixed_size=True),
    ])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

model.fit(X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, 
          min_new_neurons, validation_data=(X_test_norm, y_test))

##########################################################
Epoch 1/20
Before growing:
loss: None - accuracy: None - val_loss: 2.847022533416748 - val_accuracy: 0.0938 - penalty: 1e-06
hidden layer sizes: [20, 20, 20, 20], total units: 80
After growing:
loss: None - accuracy: None - val_loss: 2.84702205657959 - val_accuracy: 0.0938 - penalty: 1e-06
hidden layer sizes: [40, 40, 40, 40], total units: 160
Before pruning:
loss: 1.9452203512191772 - accuracy: 0.32256 - val_loss: 1.7487735748291016 - val_accuracy: 0.3872 - penalty: 1e-06
hidden layer sizes: [40, 40, 40, 40], total units: 160
After pruning:
loss: None - accuracy: None - val_loss: 1.7487735748291016 - val_accuracy: 0.3872 - penalty: 1e-06
hidden layer sizes: [40, 40, 40, 40], total units: 160
##########################################################
Epoch 2/20
Before growing:
loss: None - accuracy: None - val_loss: 1.7487735748291016 - val_accuracy: 0.3872 - penalty: 1e-06
hidden layer sizes: [40, 40, 40, 40], total units: 160

{'accuracy': [0.32256,
  0.40406,
  0.43732,
  0.46044,
  0.47494,
  0.48604,
  0.49738,
  0.50546,
  0.51368,
  0.51908,
  0.52698,
  0.53322,
  0.53872,
  0.54382,
  0.55034,
  0.55546,
  0.55652,
  0.5622,
  0.56502,
  0.56912],
 'loss': [<tf.Tensor: shape=(), dtype=float32, numpy=1.9452204>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.6932038>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.5950701>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.5310634>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.4861519>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.4510244>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.422027>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.3944101>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.3739022>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.353438>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.3362911>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.3206266>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.3053122>,
  <tf.Tensor: shape=(

In [ ]:
model = Sequential([
        Conv2D(192, filter_size=(3, 3), activation='selu', strides=(1, 1), padding='SAME', 
            regularization_penalty=0.000005, regularization_method='weighted_l1', 
            kernel_initializer='lecun_normal', input_shape=X_train_norm[0,:,:,:].shape),
        Conv2D(192, filter_size=(3, 3), activation='selu', strides=(2, 2), padding='SAME', 
            regularization_penalty=0.000005, regularization_method='weighted_l1', 
            kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.2),
        Conv2D(192, filter_size=(3, 3), activation='selu', strides=(1, 1), padding='SAME', 
            regularization_penalty=0.000005, regularization_method='weighted_l1', 
            kernel_initializer='lecun_normal'),
        Conv2D(192, filter_size=(3, 3), activation='selu', strides=(2, 2), padding='SAME', 
            regularization_penalty=0.000005, regularization_method='weighted_l1', 
            kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.5),
        Flatten(),
        # tf.keras.layers.Flatten(),
        Dense(256, activation='selu', regularization_penalty=0.000001, 
            regularization_method='weighted_l1', kernel_initializer='lecun_normal'),
        Dense(10, activation='softmax', regularization_penalty=0., 
            regularization_method=None, fixed_size=True),
    ])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

model.fit(X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, 
          min_new_neurons, validation_data=(X_test_norm, y_test))

# 74.5 % vs 71.5 % accuracy in the following two models - 3 % boost! (output deleted :P)

In [ ]:
model = Sequential([
        Conv2D(192, filter_size=(3, 3), activation='selu', strides=(1, 1), padding='SAME', 
            regularization_penalty=0.000005, regularization_method='weighted_l1', 
            kernel_initializer='lecun_normal', input_shape=X_train_norm[0,:,:,:].shape),
        Conv2D(192, filter_size=(3, 3), activation='selu', strides=(2, 2), padding='SAME', 
            regularization_penalty=0.000005, regularization_method='weighted_l1', 
            kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.2),
        Conv2D(192, filter_size=(3, 3), activation='selu', strides=(1, 1), padding='SAME', 
            regularization_penalty=0.000005, regularization_method='weighted_l1', 
            kernel_initializer='lecun_normal'),
        Conv2D(192, filter_size=(3, 3), activation='selu', strides=(2, 2), padding='SAME', 
            regularization_penalty=0.000005, regularization_method='weighted_l1', 
            kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.5),
        Flatten(),
        Dense(256, activation='selu', regularization_penalty=0.000001, 
            regularization_method='weighted_l1', kernel_initializer='lecun_normal'),
        Dense(10, activation='softmax', regularization_penalty=0., 
            regularization_method=None, fixed_size=True),
    ])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

model.fit(X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, 
          min_new_neurons, validation_data=(X_test_norm, y_test))

In [ ]:
%%time

model = Sequential([
        Conv2D(3, 96, filter_size=(3, 3), activation='selu', 
            strides=(1, 1), padding='SAME', kernel_initializer='lecun_normal', input_shape=X_train_norm[0,:,:,:].shape),
        Conv2D(96, 96, filter_size=(3, 3), activation='selu', 
            strides=(2, 2), padding='SAME', kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.2),
        Conv2D(96, 192, filter_size=(3, 3), activation='selu', 
            strides=(1, 1), padding='SAME', kernel_initializer='lecun_normal'),
        Conv2D(96, 192, filter_size=(3, 3), activation='selu', 
            strides=(2, 2), padding='SAME', kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Flatten(),
        Dense(12288, 256, activation='selu', regularization_penalty=0., 
            regularization_method=None, kernel_initializer='lecun_normal'),
        Dense(256, 10, activation='softmax', regularization_penalty=0., 
            regularization_method=None),
    ])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.fit(X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, 
          min_new_neurons, validation_data=(X_test_norm, y_test))

##########################################################
Epoch 1/25
loss: 2.1398861408233643 - accuracy: 0.39144 - val_loss: 1.3741620779037476 - val_accuracy: 0.5007 - penalty: 0.0
hidden layer sizes: [96, 96, 192, 192, 256], total units: 832
##########################################################
Epoch 2/25
loss: 1.3535252809524536 - accuracy: 0.51546 - val_loss: 1.268190860748291 - val_accuracy: 0.5486 - penalty: 0.0
hidden layer sizes: [96, 96, 192, 192, 256], total units: 832
##########################################################
Epoch 3/25
loss: 1.1914558410644531 - accuracy: 0.5725 - val_loss: 1.166611671447754 - val_accuracy: 0.5835 - penalty: 0.0
hidden layer sizes: [96, 96, 192, 192, 256], total units: 832
##########################################################
Epoch 4/25
loss: 1.0351282358169556 - accuracy: 0.63084 - val_loss: 1.087954044342041 - val_accuracy: 0.623 - penalty: 0.0
hidden layer sizes: [96, 96, 192, 192, 256], total units: 832
#####################

In [ ]:
# Temto model je na Cifar10 jiz pomerne vyladeny

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(input_shape=X_train_norm[0,:,:,:].shape, filters=96, kernel_size=(3,3), activation='selu', kernel_initializer='lecun_normal'))
model.add(tf.keras.layers.Conv2D(filters=96, kernel_size=(3,3), strides=2, activation='selu', kernel_initializer='lecun_normal'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv2D(filters=192, kernel_size=(3,3), activation='selu', kernel_initializer='lecun_normal'))
model.add(tf.keras.layers.Conv2D(filters=192, kernel_size=(3,3), strides=2, activation='selu', kernel_initializer='lecun_normal'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='selu', kernel_initializer='lecun_normal'))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
%%time

model.fit(X_train_norm, y_train, epochs=25, batch_size=256, validation_data=(X_test_norm, y_test))

Epoch 1/25
196/196 [==============================] - 9s 46ms/step - loss: 2.0276 - accuracy: 0.3617 - val_loss: 1.4487 - val_accuracy: 0.4768
Epoch 2/25
196/196 [==============================] - 9s 45ms/step - loss: 1.4076 - accuracy: 0.4938 - val_loss: 1.3187 - val_accuracy: 0.5150
Epoch 3/25
196/196 [==============================] - 9s 46ms/step - loss: 1.2646 - accuracy: 0.5483 - val_loss: 1.1942 - val_accuracy: 0.5702
Epoch 4/25
196/196 [==============================] - 9s 46ms/step - loss: 1.1380 - accuracy: 0.5945 - val_loss: 1.1212 - val_accuracy: 0.5984
Epoch 5/25
196/196 [==============================] - 9s 46ms/step - loss: 1.0255 - accuracy: 0.6347 - val_loss: 1.0245 - val_accuracy: 0.6363
Epoch 6/25
196/196 [==============================] - 9s 45ms/step - loss: 0.9390 - accuracy: 0.6672 - val_loss: 0.9999 - val_accuracy: 0.6479
Epoch 7/25
196/196 [==============================] - 9s 44ms/step - loss: 0.8667 - accuracy: 0.6939 - val_loss: 0.9372 - val_accuracy: 0.6731